# CityScan Accessibility Analysis

- This analysis will measure accessibility to only schools
- origins are derived from 100m grid blocks coming from WorldPop
- the accessibility measure is the time to the nearest facility in seconds

## Inputs:

#### origins: derived from a WorldPop raster manually using the 'Raster pixels to Points' tool in QGIS 3, and saved as a shapefile
#### destinations: The combination of POIs from Overpass as points and centroids of polygons (QGIS 3 Centoids tool). The two different point layers were merged with the QGIS 'Merge vector Layers' tool.
#### The admin AOI will be used to create a bounding box to download the road network via OSMNX


### Import libraries

In [106]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

In [107]:
# Get reference to GOSTNets
sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

In [108]:
# define output folder
inputFolder = r'cityscan_accessibility_analysis_input_folder'
# define output folder
outputFolder = r'../../../cityscan_accessibility_analysis_output_folder'
# define road graph
roadGraph = os.path.join(inputFolder, "OSM_Roads.pickle")

In [109]:
# import extent
city_extent = gpd.read_file(os.path.join(inputFolder, "Addis_AOI.shp"))

In [110]:
# This is the section where we can adjust the road speeds
if not os.path.exists(roadGraph):
    extent = box(*city_extent.total_bounds)
    G = ox.graph_from_polygon(extent, network_type = 'drive_service')
    # This is how time is calculated from the OSMNX length attribute
    G = gn.convert_network_to_time(G, 'length')
    # save the largerst subgraph
    list_of_Gs = list((nx.strongly_connected_component_subgraphs(G)))
    sz = 0
    largest_G = list_of_Gs[0]
    for g in list_of_Gs:
        curSize = g.size()
        if curSize > sz:
            largest_G = g
            sz = curSize
    G = largest_G
    print('print G')
    print(sz)
    nx.write_gpickle(G, roadGraph)
else:
    G = nx.read_gpickle(roadGraph)    
    G = gn.convert_network_to_time(G, 'length')

print G
192552


### Inspect the road graph as a shapefile

In [111]:
roads = gn.edge_gdf_from_graph(G)

In [112]:
roads[:2]

,stnode,endnode,ref,lanes,access,service,osmid,tunnel,oneway,time,bridge,highway,maxspeed,junction,width,mode,name,length,geometry
0,5532549120,5532549119,NaN,NaN,NaN,NaN,576965921,NaN,False,1.35180,NaN,unclassified,NaN,NaN,NaN,drive,NaN,7.510,"LINESTRING (38.8377067 8.887623400000001, 38.8..."
1,5532549120,5530086489,NaN,NaN,NaN,NaN,576549655,NaN,True,19.47078,NaN,unclassified,NaN,NaN,NaN,drive,NaN,108.171,"LINESTRING (38.8377067 8.887623400000001, 38.8..."


In [113]:
if not os.path.exists(outputFolder + '/OSM_Roads/OSM_Roads.shp'):
    roads['oneway'] = roads.oneway.astype(int)
    roads = roads[['oneway','length','time','mode','geometry']]
    roads.to_file(outputFolder + '/OSM_Roads')

## Create origins and destinations and snap them to the road graph

In [114]:
# insert origins
origins = gpd.read_file(os.path.join(inputFolder, "world_pop_population_pts.shp"))

In [115]:
# insert destinations
destinations = gpd.read_file(os.path.join(inputFolder, "merged_school_pts.shp"))

In [116]:
# snap the origins to the road graph
snapped_origins = gn.pandana_snap(G, origins)

In [117]:
# snap the destinations to the road graph
snapped_destinations = gn.pandana_snap(G, destinations)

In [118]:
originNodes = list(snapped_origins['NN'].unique())

In [119]:
destinationNodes = list(snapped_destinations['NN'].unique())

In [120]:
# calculate OD matrix
OD_matrix = gn.calculate_OD(G, originNodes, destinationNodes, fail_value=-1, weight='time')

In [121]:
#OD_matrix[:3]

In [122]:
closest_facility_per_origin = OD_matrix.min(axis=1)

In [123]:
closest_facility_per_origin[:5]

array([308.20653, 305.22744, 294.12147, 311.21712, 361.01709])

In [124]:
# calculate accessibility
# For each row, the closest facility is the smallest value in the row

output = snapped_origins.copy()

closest_facility_per_origin = OD_matrix.min(axis=1)

results = pd.DataFrame([originNodes, closest_facility_per_origin]).transpose()
colName = "travel_time_to_closest_facility"
results.columns = ['NN', colName]

In [125]:
results[:5]

,NN,travel_time_to_closest_facility
0,1.832108e+09,308.20653
1,5.623643e+09,305.22744
2,5.623617e+09,294.12147
3,5.623702e+09,311.21712
4,5.623592e+09,361.01709


In [126]:
output = pd.merge(output, results, on="NN")

In [127]:
output.to_csv(os.path.join(outputFolder, "addis_accessibility_schools.csv"))